In [6]:
from collections import defaultdict
import math

gt = ['A car show with people looking at cars at Becker Auto Body',	'Car that is driving pass people and becker auto body',	'A red muscle car with the license plate IDH 1969 drives past Becker Auto Body.',	'Red car that has the letters SS on the front.',	'Red car with a plate that says TDH1969 on it.']
gt = [g.lower() for g in gt]
pred = 'a car have letters becker auto body on the side side side' # 0.667
pred = 'a car have letters becker letters becker auto body on the side side side' # 0.532

In [7]:
# Step 1: split ngram and frequency
pred1gram = defaultdict(int)
gt1gram = [defaultdict(int) for _ in range(len(gt))]

for w in pred.split(): 
    pred1gram[w] += 1
for i,s in enumerate(gt):
    for w in s.split():
        gt1gram[i][w] += 1

print("Prediction 1-gram: \n%s\n" % pred1gram)
print("Ground truth 1-gram: \n%s" % gt1gram)

Prediction 1-gram: 
defaultdict(<class 'int'>, {'a': 1, 'car': 1, 'have': 1, 'letters': 2, 'becker': 2, 'auto': 1, 'body': 1, 'on': 1, 'the': 1, 'side': 3})

Ground truth 1-gram: 
[defaultdict(<class 'int'>, {'a': 1, 'car': 1, 'show': 1, 'with': 1, 'people': 1, 'looking': 1, 'at': 2, 'cars': 1, 'becker': 1, 'auto': 1, 'body': 1}), defaultdict(<class 'int'>, {'car': 1, 'that': 1, 'is': 1, 'driving': 1, 'pass': 1, 'people': 1, 'and': 1, 'becker': 1, 'auto': 1, 'body': 1}), defaultdict(<class 'int'>, {'a': 1, 'red': 1, 'muscle': 1, 'car': 1, 'with': 1, 'the': 1, 'license': 1, 'plate': 1, 'idh': 1, '1969': 1, 'drives': 1, 'past': 1, 'becker': 1, 'auto': 1, 'body.': 1}), defaultdict(<class 'int'>, {'red': 1, 'car': 1, 'that': 1, 'has': 1, 'the': 2, 'letters': 1, 'ss': 1, 'on': 1, 'front.': 1}), defaultdict(<class 'int'>, {'red': 1, 'car': 1, 'with': 1, 'a': 1, 'plate': 1, 'that': 1, 'says': 1, 'tdh1969': 1, 'on': 1, 'it.': 1})]


$$
\begin{aligned}
p_{n}=& \frac{\sum_{C \in\{\text { Candidates }\}} \sum_{n \text { -gram } \in C} \text {Count}_{clip} (n \text { -gram })}{\sum_{C^{\prime} \in\{\text { Candidates }\}} \sum_{n-g r a m^{\prime} \in C^{\prime}} \operatorname{Count}\left(n-g r a m^{\prime}\right)} .
\end{aligned}
$$

$$
\text { Count }_{\text {clip }}=\min (\text { Count }, \text { Max_Ref_Count })
$$

In [8]:
# step 2: Compute Modified Unigram Precision 
count_clip = 0
for w,fw in pred1gram.items():
    max_ref_count = gt1gram[0][w]
    for i in range(1,len(gt1gram)):
        max_ref_count = max(gt1gram[i][w], max_ref_count)

    count_clip += min(fw, max_ref_count)

mup = count_clip / len(pred.split())
print("Computed Modified Unigram Precision: ", mup)

Computed Modified Unigram Precision:  0.5714285714285714


Next, let $c$ be the length of the candidate translation and $r$ be the effective reference corpus length. We compute the brevity penalty BP,
$$
\mathrm{BP}=\left\{\begin{array}{ll}
1 & \text { if } c>r \\
e^{(1-r / c)} & \text { if } c \leq r
\end{array}\right.
$$
Then,
$$
\mathrm{BLEU}=\mathrm{BP} \cdot \exp \left(\sum_{n=1}^{N} w_{n} \log p_{n}\right)
$$
In the baseline, we use $N=4$ and uniform weights $w_{n}=1 / N$

In [9]:
# Step 3: Compute brevity penalty

# 3.1: compute effective reference length (closest length)
candlen = len(pred.split())
r = min([(abs(len(s.split()) - candlen), len(s.split())) for s in gt])[1]
print("Effective reference length r = %s"%r)
# 3.2: compute BP 
if candlen > r:
    BP = 1
else:
    BP = math.exp(1- float(r/candlen))
print("Brevity penalty BP = %s" % BP)


Effective reference length r = 15
Brevity penalty BP = 0.9310627797040228


In [10]:
# Final step: Compute BLEU
BLEU = BP * mup 
print("Computed BLEU = %.3f" % BLEU)

Computed BLEU = 0.532
